In [2]:
import pandas as pd
import numpy as np

In [3]:
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
ptm_torsion = {
    "SEP": [
        ["N", "CA", "CB", "OG"],
        ["CA", "CB", "OG", "P"],
        ["CB", "OG", "P", "O1P"]],
    "TPO": [
        ["N", "CA", "CB", "OG1"],
        ["CA", "CB", "OG1", "P"],
        ["CB", "OG1", "P", "O1P"]],
    "PTR": [
        ["N", "CA", "CB", "CG"],
        ["CA", "CB", "CG", "CD1"],
        ["CE1", "CZ", "OH", "P"],
        ["CZ", "OH", "P", "O1P"]],
    "M3L": [
        ["N", "CA", "CB", "CG"],
        ["CA", "CB", "CG", "CD"],
        ["CB", "CG", "CD", "CE"],
        ["CG", "CD", "CE", "NZ"],
        ["CD", "CE", "NZ", "CM1"]
        ],
    "ALY": [
        ["N", "CA", "CB", "CG"],
        ["CA", "CB", "CG", "CD"],
        ["CB", "CG", "CD", "CE"],
        ["CG", "CD", "CE", "NZ"],
        ["CD", "CE", "NZ", "CH"],
        ],
}

nchis = {k: len(v) for k, v in ptm_torsion.items()}



In [8]:
def polar_histogram(data, nbins=36):
    if len(data) == 0:
        bins = np.linspace(-np.pi, np.pi, nbins, endpoint=False)
        return np.zeros(nbins - 1), np.zeros(nbins - 1), (bins[:-1] + bins[1:]) / 2
    hist, bins = np.histogram(data, bins=np.linspace(-np.pi, np.pi, nbins+1))
    sqrt_hist = np.sqrt(hist)/np.max(np.sqrt(hist))
    return sqrt_hist, hist/np.sum(hist) * 100, (bins[:-1] + bins[1:]) / 2

In [9]:
from scipy.special import i0

def vonmises_kde(data, kappa, n_bins=36):
    x = np.linspace(-np.pi, np.pi, n_bins)
    # integrate vonmises kernels
    kde = np.exp(kappa * np.cos(x[:, None] - data[None, :])).sum(1) / (2*np.pi*i0(kappa))
    kde /= np.trapz(kde, x=x)
    return x, kde

PTM data analysis; SEP as an example for visualizing histograms across backbone regions

In [14]:
# read ptm dataframes
ptm_data = []
for ptm in ptm_torsion:
    ptm_data.append(pd.read_csv(f"../data/{ptm}_df.csv"))
ptm_data = pd.concat(ptm_data)


In [ ]:
resn = "SEP"
data = ptm_data[ptm_data.residue==resn]
# Normalize the data to [0, 1] for color mapping
norm = colors.Normalize(0, 100)
cmap = cm.winter_r

fig, ax = plt.subplots(4, 4, figsize=(10, 10), subplot_kw={'projection': 'polar'})
for j in range(4):
    if j == 0: 
        # backbone independent
        angles = data
        xlabel = "bb. indp."
    elif j == 1:
        # helix
        angles = data[(-105 <= data.phi).values & (data.phi < -45).values & \
                            (-60 <= data.psi).values & (data.psi < 30).values]
        xlabel = r"$\alpha$-helix"
    elif j == 2:
        # beta
        angles = data[(data.phi.values < -105) & (data.psi.values > 90)]
        xlabel = r"$\beta$-sheet"
    else:
        # PII
        angles = data[(-105 <= data.phi).values  & (data.phi < -45).values & \
                        (data.psi.values > 105)]
        xlabel = "polyproline"
    for i in range(4):
        ax[i, j].set_theta_zero_location('E')
        ax[i, j].yaxis.grid(False)
        ax[i, j].set_yticks([])
        if i < 3:
            ax[i, j].set_xlabel(None)
        if j > 0:
            ax[i, j].set_ylabel(None)

        sqrth, h, b = polar_histogram(angles[f"chi{i+1}"].apply(np.radians).values, nbins=36)
        if i in [1, 2] and resn in ["PTR", "TYR"]:
            sqrth, h, b = polar_histogram(angles[f"chi{i+1}"].apply(np.radians).values, nbins=24)
            mask = np.logical_and(np.pi/4 < b, b <= 3*np.pi/4)
            ax[i, j].bar(b[mask], sqrth[mask], width=np.radians(15), alpha=0.7, color=cmap(norm(h[mask].sum())))
            mask = np.logical_and(-3*np.pi/4 < b, b <= -np.pi/4)
            ax[i, j].bar(b[mask], sqrth[mask], width=np.radians(15), alpha=0.7, color=cmap(norm(h[mask].sum())))
            mask = np.logical_and(-np.pi/4 < b, b <= np.pi/4)
            ax[i, j].bar(b[mask], sqrth[mask], width=np.radians(15), alpha=0.7, color=cmap(norm(h[mask].sum())))
            mask = np.logical_or(-3*np.pi/4 >= b, b > 3*np.pi/4)
            ax[i, j].bar(b[mask], sqrth[mask], width=np.radians(15), alpha=0.7, color=cmap(norm(h[mask].sum())))

            ax[i, j].set_xticks([np.pi/4, 3*np.pi/4, 5*np.pi/4, 7*np.pi/4], [r"$45^\circ$", r"$135^\circ$", r"$-135^\circ$", r"$-45^\circ$"])  
            continue
        elif i == 4 and resn == "ALY":
            sqrth, h, b = polar_histogram(angles[f"chi{i+1}"].apply(np.radians).values, nbins=36)
            mask = np.logical_and(np.pi/6 < b, b <= np.pi/2)
            ax[i, j].bar(b[mask], sqrth[mask], width=np.radians(10), alpha=0.7, color=cmap(norm(h[mask].sum())))
            mask = np.logical_and(np.pi/2 < b, b <= 5*np.pi/6)
            ax[i, j].bar(b[mask], sqrth[mask], width=np.radians(10), alpha=0.7, color=cmap(norm(h[mask].sum())))
            mask = np.logical_and(-np.pi/6 < b, b <= np.pi/6)
            ax[i, j].bar(b[mask], sqrth[mask], width=np.radians(10), alpha=0.7, color=cmap(norm(h[mask].sum())))
            mask = np.logical_and(-np.pi/2 < b, b <= -np.pi/6)
            ax[i, j].bar(b[mask], sqrth[mask], width=np.radians(10), alpha=0.7, color=cmap(norm(h[mask].sum())))
            mask = np.logical_and(-5*np.pi/6 < b, b <= -np.pi/2)
            ax[i, j].bar(b[mask], sqrth[mask], width=np.radians(10), alpha=0.7, color=cmap(norm(h[mask].sum())))
            mask = np.logical_or(-5*np.pi/6 >= b, b > 5*np.pi/6)
            ax[i, j].bar(b[mask], sqrth[mask], width=np.radians(10), alpha=0.7, color=cmap(norm(h[mask].sum())))

            ax[i, j].set_xticks([np.pi/6, np.pi/2, 5*np.pi/6, 7*np.pi/6, 3*np.pi/2, 11*np.pi/6], 
                                [r"$30^\circ$", r"$90^\circ$", r"$150^\circ$", r"$-150^\circ$", r"$-90^\circ$", r"$-30^\circ$"])  
            continue
        
        mask = np.logical_and(0 < b, b <= 2*np.pi/3)
        ax[i, j].bar(b[mask], sqrth[mask], width=np.radians(10), alpha=0.7, color=cmap(norm(h[mask].sum())))
        mask = np.logical_and(-2*np.pi/3 < b, b <= 0)
        ax[i, j].bar(b[mask], sqrth[mask], width=np.radians(10), alpha=0.7, color=cmap(norm(h[mask].sum())))
        mask = np.logical_or(2*np.pi/3 < b, b <= -2*np.pi/3)
        ax[i, j].bar(b[mask], sqrth[mask], width=np.radians(10), alpha=0.7, color=cmap(norm(h[mask].sum())))
        ax[i, j].set_xticks([0, 2*np.pi/3, 4*np.pi/3], [r"$0^\circ$", r"$120^\circ$", r"$-120^\circ$"])  

# Add colorbar
#sm = cm.ScalarMappable(cmap=cmap, norm=norm)
#sm.set_array([])
#cbar = plt.colorbar(sm)
#cbar.ax.set_ylabel('Rotamer population (%)')  

    

Visualize chi1 distributions on ramachandran plots

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(14, 3))
norm = colors.Normalize(-180, 180)
cmap = cm.twilight_shifted
ax[0].scatter(ptm_data[ptm_data.residue=="SEP"].phi, ptm_data[ptm_data.residue=="SEP"].psi, 
                c=ptm_data[ptm_data.residue=="SEP"].chi1, cmap=cmap, alpha=0.7)
ax[1].scatter(ptm_data[ptm_data.residue=="TPO"].phi, ptm_data[ptm_data.residue=="TPO"].psi, 
                c=ptm_data[ptm_data.residue=="TPO"].chi1, cmap=cmap, alpha=0.7)
ax[2].scatter(ptm_data[ptm_data.residue=="PTR"].phi, ptm_data[ptm_data.residue=="PTR"].psi, 
                c=ptm_data[ptm_data.residue=="PTR"].chi1, cmap=cmap, alpha=0.7)
ax[3].scatter(ptm_data[ptm_data.residue=="M3L"].phi, ptm_data[ptm_data.residue=="M3L"].psi, 
                c=ptm_data[ptm_data.residue=="M3L"].chi1, cmap=cmap, alpha=0.7)

for i in range(4):
    ax[i].set_xlim(-180, 180)
    ax[i].set_ylim(-180, 180)
    # Highlight grid lines at x = [-120, 0, 120] and y = [-120, 0, 120] with magenta
    for val in [30, 90, 150, -30, -90, -150]:
        ax[i].axvline(x=val, color='grey', linestyle='--', linewidth=0.6)
        ax[i].axhline(y=val, color='grey', linestyle='--', linewidth=0.6)
    if i > 0:
        ax[i].set_yticks([])

Helper function for returning backbone angles of residues forming local hydrogen bonds

In [29]:
from Bio.PDB import calc_dihedral, PDBParser

parser = PDBParser()
def phosphate_hbond(pdb, ptm):
    struct = parser.get_structure("t", pdb)
    backbone = []
    for chain in struct[0]:
        for res in chain:
            if res.get_resname() == ptm:
                try:
                    dist = min(res["N"] - res[f"O{i+1}P"] for i in range(3))
                    phi = calc_dihedral(chain[res.id[1] - 1]["C"].get_vector(), 
                            res["N"].get_vector(), res["CA"].get_vector(), res["C"].get_vector())
                    psi = calc_dihedral(res["N"].get_vector(), 
                            res["CA"].get_vector(), res["C"].get_vector(), chain[res.id[1] + 1]["N"].get_vector())
                    chi1 = calc_dihedral(*(res[atom].get_vector() for atom in ptm_torsion[ptm][0]))
                    chi2 = calc_dihedral(*(res[atom].get_vector() for atom in ptm_torsion[ptm][1]))
                    if dist <= 3.5 or min(chain[res.id[1] + 1]["N"] - res[f"O{i+1}P"] for i in range(3)) <= 3.5:
                        backbone.append((np.degrees(phi), np.degrees(psi), np.degrees(chi1), np.degrees(chi2)))
                except KeyError as e:
                    continue
                    #raise KeyError(pdb, chain.id, res.id)
    return backbone
                